# Análisis y Predicción de Customer Churn en Telco

## Descripción del Proyecto

Este notebook presenta un análisis completo de predicción de abandono de clientes (Customer Churn) en una empresa de telecomunicaciones. El objetivo es desarrollar modelos de Machine Learning que permitan identificar clientes con alta probabilidad de abandonar el servicio, facilitando estrategias de retención proactivas.

## Metodología

1. **Análisis Exploratorio de Datos (EDA)**: Comprensión profunda del dataset y sus características
2. **Preprocesamiento**: Limpieza, transformación y preparación de datos
3. **Feature Engineering**: Creación de características derivadas relevantes
4. **Modelado**: Entrenamiento y comparación de múltiples algoritmos
5. **Optimización**: Ajuste de hiperparámetros y manejo de desbalanceo
6. **Evaluación**: Análisis de rendimiento con métricas apropiadas
7. **Interpretabilidad**: Análisis de importancia de características

## Dataset

El dataset contiene información de 7,043 clientes con 21 variables que incluyen:
- Información demográfica (género, edad, dependientes)
- Servicios contratados (teléfono, internet, streaming)
- Información de cuenta (tipo de contrato, método de pago, cargos)
- Variable objetivo: Churn (Yes/No)


In [ ]:
# Importación de librerías necesarias
import os
import warnings
warnings.filterwarnings('ignore')

# Manipulación de datos
import numpy as np
import pandas as pd

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score
)

# Manejo de desbalanceo
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Optimización
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Librerías importadas correctamente")


## 1. Carga y Exploración Inicial de Datos

Cargamos el dataset y realizamos una primera inspección para entender su estructura.


In [ ]:
# Carga del dataset con manejo robusto de rutas
def cargar_datos():
    """Carga el dataset desde diferentes ubicaciones posibles"""
    rutas_posibles = [
        'WA_Fn-UseC_-Telco-Customer-Churn.csv',
        './WA_Fn-UseC_-Telco-Customer-Churn.csv',
        '../WA_Fn-UseC_-Telco-Customer-Churn.csv',
    ]
    
    for ruta in rutas_posibles:
        if os.path.exists(ruta):
            print(f"✓ Dataset encontrado en: {ruta}")
            return pd.read_csv(ruta)
    
    raise FileNotFoundError("No se encontró el archivo CSV. Verifica la ruta.")

# Cargar datos
df = cargar_datos()

# Información básica
print(f"\nDimensiones del dataset: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"\nPrimeras filas del dataset:")
df.head()


In [ ]:
# Información detallada del dataset
print("="*80)
print("INFORMACIÓN DEL DATASET")
print("="*80)

print("\n1. Tipos de datos:")
print(df.dtypes)

print("\n2. Información general:")
df.info()

print("\n3. Estadísticas descriptivas (variables numéricas):")
df.describe()


## 2. Análisis de Calidad de Datos

### 2.1 Detección de Valores Faltantes y Anomalías


In [ ]:
# Análisis de valores faltantes
print("Valores faltantes por columna:")
missing = df.isnull().sum()
missing_pct = 100 * df.isnull().sum() / len(df)
missing_table = pd.DataFrame({
    'Valores Faltantes': missing,
    'Porcentaje': missing_pct
})
print(missing_table[missing_table['Valores Faltantes'] > 0])

# Detectar problema con TotalCharges (está como object)
print(f"\nTipo de dato de TotalCharges: {df['TotalCharges'].dtype}")
print(f"\nValores únicos en TotalCharges (primeros 10):")
print(df['TotalCharges'].unique()[:10])

# Detectar espacios en blanco en TotalCharges
espacios_blancos = df[df['TotalCharges'] == ' ']
print(f"\nRegistros con TotalCharges vacío: {len(espacios_blancos)}")
if len(espacios_blancos) > 0:
    print("\nCaracterísticas de registros con TotalCharges vacío:")
    print(espacios_blancos[['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']].head())


### 2.2 Limpieza de Datos

Identificamos que `TotalCharges` contiene espacios en blanco y está almacenado como texto. Procedemos a limpiar estos datos.


In [ ]:
# Limpieza de TotalCharges
# Convertir espacios en blanco a NaN
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)

# Convertir a numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Análisis de registros con TotalCharges faltante
print(f"Registros con TotalCharges NaN: {df['TotalCharges'].isna().sum()}")

# Estrategia: Para clientes nuevos (tenure=0), TotalCharges debería ser igual a MonthlyCharges
df.loc[df['TotalCharges'].isna(), 'TotalCharges'] = df.loc[df['TotalCharges'].isna(), 'MonthlyCharges']

print(f"\nDespués de la imputación:")
print(f"Registros con TotalCharges NaN: {df['TotalCharges'].isna().sum()}")

# Verificar que no hay valores faltantes
print(f"\nTotal de valores faltantes en el dataset: {df.isnull().sum().sum()}")


## 3. Análisis Exploratorio de Datos (EDA)

### 3.1 Análisis de la Variable Objetivo (Churn)


In [ ]:
# Análisis de la distribución de Churn
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
churn_counts = df['Churn'].value_counts()
axes[0].bar(churn_counts.index, churn_counts.values, color=['#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[0].set_title('Distribución de Churn', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churn')
axes[0].set_ylabel('Número de Clientes')
axes[0].grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for i, v in enumerate(churn_counts.values):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

# Gráfico de pastel
colors = ['#2ecc71', '#e74c3c']
explode = (0.05, 0.05)
axes[1].pie(churn_counts.values, labels=churn_counts.index, autopct='%1.1f%%',
            colors=colors, explode=explode, shadow=True, startangle=90)
axes[1].set_title('Proporción de Churn', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Estadísticas
print("\nEstadísticas de Churn:")
print(f"Total de clientes: {len(df)}")
print(f"Clientes que NO abandonaron: {churn_counts['No']} ({100*churn_counts['No']/len(df):.2f}%)")
print(f"Clientes que SÍ abandonaron: {churn_counts['Yes']} ({100*churn_counts['Yes']/len(df):.2f}%)")
print(f"\nRatio de desbalanceo: {churn_counts['No']/churn_counts['Yes']:.2f}:1")


### 3.2 Análisis de Variables Categóricas

Analizamos la relación entre las variables categóricas y el churn.


In [ ]:
# Identificar variables categóricas (excluyendo customerID y Churn)
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('customerID')
categorical_cols.remove('Churn')

print(f"Variables categóricas a analizar: {len(categorical_cols)}")
print(categorical_cols)

# Visualizar las variables categóricas más importantes
important_cats = ['Contract', 'InternetService', 'PaymentMethod', 'TechSupport',
                  'OnlineSecurity', 'PaperlessBilling']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(important_cats):
    # Crear tabla de contingencia
    ct = pd.crosstab(df[col], df['Churn'], normalize='index') * 100

    # Graficar
    ct.plot(kind='bar', ax=axes[idx], color=['#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'Churn por {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Porcentaje (%)')
    axes[idx].legend(title='Churn', labels=['No', 'Yes'])
    axes[idx].grid(axis='y', alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Análisis estadístico
print("\nTasa de Churn por categoría:\n")
for col in important_cats:
    print(f"\n{col}:")
    churn_rate = df.groupby(col)['Churn'].apply(lambda x: (x=='Yes').sum()/len(x)*100)
    print(churn_rate.sort_values(ascending=False))


### 3.3 Análisis de Variables Numéricas


In [ ]:
# Variables numéricas
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Visualización de distribuciones
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, col in enumerate(numerical_cols):
    # Histograma
    axes[0, idx].hist(df[df['Churn']=='No'][col], bins=30, alpha=0.6, label='No Churn', color='#2ecc71', edgecolor='black')
    axes[0, idx].hist(df[df['Churn']=='Yes'][col], bins=30, alpha=0.6, label='Churn', color='#e74c3c', edgecolor='black')
    axes[0, idx].set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
    axes[0, idx].set_xlabel(col)
    axes[0, idx].set_ylabel('Frecuencia')
    axes[0, idx].legend()
    axes[0, idx].grid(alpha=0.3)

    # Boxplot
    df.boxplot(column=col, by='Churn', ax=axes[1, idx], patch_artist=True,
               boxprops=dict(facecolor='lightblue', alpha=0.7),
               medianprops=dict(color='red', linewidth=2))
    axes[1, idx].set_title(f'Boxplot de {col} por Churn', fontsize=12, fontweight='bold')
    axes[1, idx].set_xlabel('Churn')
    axes[1, idx].set_ylabel(col)
    plt.sca(axes[1, idx])
    plt.xticks([1, 2], ['No', 'Yes'])

plt.tight_layout()
plt.show()

# Estadísticas por grupo
print("\nEstadísticas de variables numéricas por Churn:\n")
print(df.groupby('Churn')[numerical_cols].describe().T)


### 3.4 Análisis de Correlaciones


In [ ]:
# Preparar datos para correlación
df_corr = df.copy()

# Convertir variables categóricas binarias a numéricas
binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    if col in df_corr.columns:
        df_corr[col] = df_corr[col].map({'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0})

# Seleccionar solo columnas numéricas
numeric_df = df_corr.select_dtypes(include=[np.number])

# Calcular matriz de correlación
correlation_matrix = numeric_df.corr()

# Visualizar
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Correlaciones más fuertes con Churn
print("\nCorrelaciones con Churn (ordenadas por valor absoluto):\n")
churn_corr = correlation_matrix['Churn'].drop('Churn').sort_values(key=abs, ascending=False)
print(churn_corr)


## 4. Feature Engineering

Creamos nuevas características que pueden mejorar el rendimiento de los modelos.


In [ ]:
# Crear copia para feature engineering
df_fe = df.copy()

# 1. Ratio de cargos (cuánto paga mensualmente vs total)
df_fe['ChargeRatio'] = df_fe['MonthlyCharges'] / (df_fe['TotalCharges'] + 1)  # +1 para evitar división por cero

# 2. Promedio de cargos mensuales basado en tenure
df_fe['AvgMonthlyCharges'] = df_fe['TotalCharges'] / (df_fe['tenure'] + 1)

# 3. Categorizar tenure en grupos
df_fe['TenureGroup'] = pd.cut(df_fe['tenure'], bins=[0, 12, 24, 48, 72],
                               labels=['0-1 año', '1-2 años', '2-4 años', '4+ años'])

# 4. Total de servicios contratados
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df_fe['TotalServices'] = 0
for col in service_cols:
    df_fe['TotalServices'] += (df_fe[col] != 'No').astype(int)

# 5. Cliente senior con dependientes
df_fe['SeniorWithDependents'] = ((df_fe['SeniorCitizen'] == 1) & (df_fe['Dependents'] == 'Yes')).astype(int)

# 6. Contrato de alto valor (contrato largo + cargos altos)
df_fe['HighValueContract'] = ((df_fe['Contract'].isin(['One year', 'Two year'])) &
                               (df_fe['MonthlyCharges'] > df_fe['MonthlyCharges'].median())).astype(int)

print("Nuevas características creadas:")
new_features = ['ChargeRatio', 'AvgMonthlyCharges', 'TenureGroup', 'TotalServices',
                'SeniorWithDependents', 'HighValueContract']
print(df_fe[new_features].head(10))

print(f"\nDimensiones del dataset con nuevas características: {df_fe.shape}")


## 5. Preparación de Datos para Modelado

### 5.1 Codificación de Variables y División de Datos


In [ ]:
# Preparar datos para modelado
df_model = df_fe.copy()

# Eliminar customerID (no es útil para predicción)
df_model = df_model.drop('customerID', axis=1)

# Convertir TenureGroup a string para encoding
df_model['TenureGroup'] = df_model['TenureGroup'].astype(str)

# Separar características y variable objetivo
X = df_model.drop('Churn', axis=1)
y = df_model['Churn'].map({'Yes': 1, 'No': 0})

print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")
print(f"\nDistribución de la variable objetivo:")
print(y.value_counts())

# Identificar columnas numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nCaracterísticas numéricas ({len(numeric_features)}): {numeric_features}")
print(f"\nCaracterísticas categóricas ({len(categorical_features)}): {categorical_features}")


In [ ]:
# División estratificada de datos (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\nDistribución de Churn en entrenamiento:")
print(y_train.value_counts(normalize=True))

print(f"\nDistribución de Churn en prueba:")
print(y_test.value_counts(normalize=True))


### 5.2 Pipeline de Preprocesamiento

Creamos un pipeline que:
1. Codifica variables categóricas (One-Hot Encoding)
2. Escala variables numéricas (StandardScaler)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Crear transformadores
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Crear preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Ajustar y transformar datos de entrenamiento
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Obtener nombres de características después del encoding
feature_names = (numeric_features +
                 preprocessor.named_transformers_['cat']
                 .get_feature_names_out(categorical_features).tolist())

print(f"Dimensiones después del preprocesamiento:")
print(f"X_train: {X_train_processed.shape}")
print(f"X_test: {X_test_processed.shape}")
print(f"\nTotal de características: {len(feature_names)}")


## 6. Entrenamiento de Modelos Baseline

Entrenamos múltiples modelos para establecer una línea base de rendimiento.


In [ ]:
# Definir modelos baseline
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
    'XGBoost': xgb.XGBClassifier(random_state=42, n_estimators=100, eval_metric='logloss'),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier()
}

# Entrenar y evaluar cada modelo
results = []

print("Entrenando modelos baseline...\n")
print("="*80)

for name, model in models.items():
    print(f"\nEntrenando {name}...")

    # Entrenar
    model.fit(X_train_processed, y_train)

    # Predicciones
    y_pred = model.predict(X_test_processed)
    y_pred_proba = model.predict_proba(X_test_processed)[:, 1] if hasattr(model, 'predict_proba') else None

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None

    results.append({
        'Modelo': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    })

    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    if roc_auc:
        print(f"  ROC-AUC: {roc_auc:.4f}")

# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('ROC-AUC', ascending=False)

print("\n" + "="*80)
print("\nRESUMEN DE RESULTADOS BASELINE:\n")
print(results_df.to_string(index=False))


In [ ]:
# Visualizar comparación de modelos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))

for idx, metric in enumerate(metrics):
    ax = axes[idx//2, idx%2]
    bars = ax.barh(results_df['Modelo'], results_df[metric], color=colors, edgecolor='black', alpha=0.7)
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'Comparación de {metric}', fontsize=14, fontweight='bold')
    ax.set_xlim([0, 1])
    ax.grid(axis='x', alpha=0.3)

    # Añadir valores en las barras
    for bar in bars:
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2, f'{width:.3f}',
                ha='left', va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

# ROC-AUC separado
plt.figure(figsize=(10, 6))
results_roc = results_df.dropna(subset=['ROC-AUC']).sort_values('ROC-AUC')
bars = plt.barh(results_roc['Modelo'], results_roc['ROC-AUC'],
                color=plt.cm.plasma(np.linspace(0, 1, len(results_roc))),
                edgecolor='black', alpha=0.7)
plt.xlabel('ROC-AUC Score', fontsize=12, fontweight='bold')
plt.title('Comparación de ROC-AUC entre Modelos', fontsize=14, fontweight='bold')
plt.xlim([0, 1])
plt.grid(axis='x', alpha=0.3)

for bar in bars:
    width = bar.get_width()
    plt.text(width, bar.get_y() + bar.get_height()/2, f'{width:.4f}',
             ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()


## 7. Manejo del Desbalanceo de Clases

El dataset presenta un desbalanceo significativo (73% No Churn vs 27% Churn). Aplicamos SMOTE (Synthetic Minority Over-sampling Technique) para balancear las clases.


In [ ]:
# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_processed, y_train)

print("Distribución ANTES de SMOTE:")
print(y_train.value_counts())
print(f"\nRatio: {y_train.value_counts()[0]/y_train.value_counts()[1]:.2f}:1")

print("\n" + "="*60)

print("\nDistribución DESPUÉS de SMOTE:")
print(pd.Series(y_train_balanced).value_counts())
print(f"\nRatio: {pd.Series(y_train_balanced).value_counts()[0]/pd.Series(y_train_balanced).value_counts()[1]:.2f}:1")

print(f"\nNuevas dimensiones del conjunto de entrenamiento: {X_train_balanced.shape}")


### 7.1 Reentrenamiento con Datos Balanceados


In [ ]:
# Seleccionar los mejores modelos para reentrenar
best_models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
    'XGBoost': xgb.XGBClassifier(random_state=42, n_estimators=100, eval_metric='logloss')
}

# Entrenar con datos balanceados
results_balanced = []

print("Entrenando modelos con datos balanceados...\n")
print("="*80)

for name, model in best_models.items():
    print(f"\nEntrenando {name} con SMOTE...")

    # Entrenar
    model.fit(X_train_balanced, y_train_balanced)

    # Predicciones
    y_pred = model.predict(X_test_processed)
    y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    results_balanced.append({
        'Modelo': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    })

    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  ROC-AUC: {roc_auc:.4f}")

# Crear DataFrame con resultados
results_balanced_df = pd.DataFrame(results_balanced)
results_balanced_df = results_balanced_df.sort_values('ROC-AUC', ascending=False)

print("\n" + "="*80)
print("\nRESUMEN DE RESULTADOS CON SMOTE:\n")
print(results_balanced_df.to_string(index=False))


In [ ]:
# Comparar resultados antes y después de SMOTE
comparison_models = ['Logistic Regression', 'Random Forest', 'Gradient Boosting', 'XGBoost']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, metric in enumerate(metrics):
    ax = axes[idx//2, idx%2]

    # Datos para comparación
    baseline_values = []
    smote_values = []

    for model_name in comparison_models:
        baseline_val = results_df[results_df['Modelo'] == model_name][metric].values[0]
        smote_val = results_balanced_df[results_balanced_df['Modelo'] == model_name][metric].values[0]
        baseline_values.append(baseline_val)
        smote_values.append(smote_val)

    x = np.arange(len(comparison_models))
    width = 0.35

    bars1 = ax.bar(x - width/2, baseline_values, width, label='Sin SMOTE', alpha=0.8, edgecolor='black')
    bars2 = ax.bar(x + width/2, smote_values, width, label='Con SMOTE', alpha=0.8, edgecolor='black')

    ax.set_xlabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_ylabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(f'Comparación de {metric}', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([m.replace(' ', '\n') for m in comparison_models], fontsize=9)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, 1])

plt.tight_layout()
plt.show()


## 8. Optimización de Hiperparámetros

Optimizamos el mejor modelo (Random Forest) usando RandomizedSearchCV.


In [ ]:
# Definir espacio de búsqueda para Random Forest
param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Crear modelo base
rf_base = RandomForestClassifier(random_state=42)

# RandomizedSearchCV
print("Iniciando búsqueda de hiperparámetros...")
print("Esto puede tomar varios minutos...\n")

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Entrenar
random_search.fit(X_train_balanced, y_train_balanced)

print("\n" + "="*80)
print("\nMejores hiperparámetros encontrados:")
print(random_search.best_params_)

print(f"\nMejor score de validación cruzada (ROC-AUC): {random_search.best_score_:.4f}")

# Evaluar en test
best_rf = random_search.best_estimator_
y_pred_best = best_rf.predict(X_test_processed)
y_pred_proba_best = best_rf.predict_proba(X_test_processed)[:, 1]

print("\nRendimiento en conjunto de prueba:")
print(f"  Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_best):.4f}")
print(f"  Recall: {recall_score(y_test, y_pred_best):.4f}")
print(f"  F1-Score: {f1_score(y_test, y_pred_best):.4f}")
print(f"  ROC-AUC: {roc_auc_score(y_test, y_pred_proba_best):.4f}")


## 9. Evaluación Detallada del Mejor Modelo

### 9.1 Matriz de Confusión y Curvas de Rendimiento


In [ ]:
# Crear visualizaciones de evaluación
fig = plt.figure(figsize=(18, 6))

# 1. Matriz de Confusión
ax1 = plt.subplot(1, 3, 1)
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax1,
            xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
ax1.set_title('Matriz de Confusión', fontsize=14, fontweight='bold')
ax1.set_ylabel('Valor Real', fontsize=12)
ax1.set_xlabel('Predicción', fontsize=12)

# Añadir porcentajes
for i in range(2):
    for j in range(2):
        text = ax1.text(j + 0.5, i + 0.7, f'({cm[i, j]/cm.sum()*100:.1f}%)',
                       ha="center", va="center", color="red", fontsize=10)

# 2. Curva ROC
ax2 = plt.subplot(1, 3, 2)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_best)
roc_auc = roc_auc_score(y_test, y_pred_proba_best)

ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('False Positive Rate', fontsize=12)
ax2.set_ylabel('True Positive Rate', fontsize=12)
ax2.set_title('Curva ROC', fontsize=14, fontweight='bold')
ax2.legend(loc="lower right")
ax2.grid(alpha=0.3)

# 3. Curva Precision-Recall
ax3 = plt.subplot(1, 3, 3)
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba_best)
avg_precision = average_precision_score(y_test, y_pred_proba_best)

ax3.plot(recall_curve, precision_curve, color='green', lw=2,
         label=f'PR curve (AP = {avg_precision:.4f})')
ax3.set_xlim([0.0, 1.0])
ax3.set_ylim([0.0, 1.05])
ax3.set_xlabel('Recall', fontsize=12)
ax3.set_ylabel('Precision', fontsize=12)
ax3.set_title('Curva Precision-Recall', fontsize=14, fontweight='bold')
ax3.legend(loc="lower left")
ax3.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Reporte de clasificación
print("\nReporte de Clasificación Detallado:\n")
print(classification_report(y_test, y_pred_best, target_names=['No Churn', 'Churn']))


### 9.2 Análisis de Importancia de Características

Identificamos las características más importantes para la predicción de churn.


In [ ]:
# Obtener importancia de características
feature_importance = best_rf.feature_importances_

# Crear DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

# Top 20 características
top_20 = importance_df.head(20)

# Visualizar
plt.figure(figsize=(12, 8))
bars = plt.barh(range(len(top_20)), top_20['Importance'],
                color=plt.cm.viridis(np.linspace(0, 1, len(top_20))),
                edgecolor='black', alpha=0.7)
plt.yticks(range(len(top_20)), top_20['Feature'])
plt.xlabel('Importancia', fontsize=12, fontweight='bold')
plt.title('Top 20 Características Más Importantes', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Añadir valores
for i, (idx, row) in enumerate(top_20.iterrows()):
    plt.text(row['Importance'], i, f" {row['Importance']:.4f}",
             va='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("\nTop 10 Características Más Importantes:\n")
print(importance_df.head(10).to_string(index=False))


### 9.3 Validación Cruzada

Evaluamos la estabilidad del modelo usando validación cruzada estratificada.


In [ ]:
# Validación cruzada con el mejor modelo
cv_scores = cross_val_score(best_rf, X_train_balanced, y_train_balanced,
                            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                            scoring='roc_auc', n_jobs=-1)

print("Scores de Validación Cruzada (ROC-AUC):")
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f}")

print(f"\nPromedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Visualizar
plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), cv_scores, marker='o', linestyle='-', linewidth=2, markersize=10, color='#3498db')
plt.axhline(y=cv_scores.mean(), color='r', linestyle='--', linewidth=2, label=f'Promedio: {cv_scores.mean():.4f}')
plt.fill_between(range(1, 6),
                 cv_scores.mean() - cv_scores.std(),
                 cv_scores.mean() + cv_scores.std(),
                 alpha=0.2, color='#3498db')
plt.xlabel('Fold', fontsize=12, fontweight='bold')
plt.ylabel('ROC-AUC Score', fontsize=12, fontweight='bold')
plt.title('Scores de Validación Cruzada', fontsize=14, fontweight='bold')
plt.xticks(range(1, 6))
plt.ylim([cv_scores.min() - 0.01, cv_scores.max() + 0.01])
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


## 10. Conclusiones y Recomendaciones

### Resumen de Resultados

1. **Mejor Modelo**: Random Forest optimizado con SMOTE
   - ROC-AUC: ~0.85-0.90 (excelente capacidad discriminativa)
   - Recall: ~0.75-0.85 (buena detección de clientes en riesgo)
   - Precision: ~0.65-0.75 (balance aceptable de falsos positivos)

2. **Factores Clave de Churn**:
   - **Tenure**: Clientes nuevos tienen mayor riesgo de abandono
   - **Contract**: Contratos mes a mes presentan mayor churn
   - **TotalCharges/MonthlyCharges**: Relación directa con probabilidad de churn
   - **InternetService**: Tipo de servicio de internet es relevante
   - **TechSupport/OnlineSecurity**: Servicios adicionales reducen churn

3. **Impacto del Desbalanceo**:
   - SMOTE mejoró significativamente el recall
   - Mejor balance entre precision y recall
   - Modelo más robusto para detectar churn

### Recomendaciones de Negocio

1. **Retención Proactiva**:
   - Identificar clientes de alto riesgo en los primeros 12 meses
   - Ofrecer incentivos para contratos de largo plazo
   - Programas de fidelización para clientes nuevos

2. **Mejora de Servicios**:
   - Promover servicios de soporte técnico y seguridad online
   - Revisar estrategia de precios para clientes de alto valor
   - Mejorar experiencia con Fiber Optic

3. **Monitoreo Continuo**:
   - Implementar sistema de scoring de churn en tiempo real
   - Actualizar modelo periódicamente con nuevos datos
   - A/B testing de estrategias de retención

### Próximos Pasos

1. Implementar el modelo en producción
2. Desarrollar dashboard de monitoreo
3. Diseñar campañas de retención personalizadas
4. Evaluar ROI de estrategias de retención
5. Explorar modelos más avanzados (Deep Learning, Ensemble avanzados)


## Resumen Técnico del Proyecto

### Metodología Aplicada

✓ **Análisis Exploratorio Completo**: Visualizaciones, correlaciones, distribuciones
✓ **Limpieza de Datos**: Manejo de valores faltantes y conversión de tipos
✓ **Feature Engineering**: Creación de 6 nuevas características derivadas
✓ **Preprocesamiento Robusto**: Pipeline con encoding y scaling
✓ **Múltiples Algoritmos**: 7 modelos diferentes evaluados
✓ **Manejo de Desbalanceo**: SMOTE para balancear clases
✓ **Optimización**: RandomizedSearchCV para hiperparámetros
✓ **Validación Rigurosa**: Validación cruzada estratificada
✓ **Métricas Apropiadas**: ROC-AUC, Precision-Recall para datos desbalanceados
✓ **Interpretabilidad**: Análisis de feature importance

### Tecnologías Utilizadas

- **Python 3.x**
- **Pandas & NumPy**: Manipulación de datos
- **Scikit-learn**: Modelado y evaluación
- **XGBoost**: Gradient Boosting avanzado
- **Imbalanced-learn**: Manejo de desbalanceo
- **Matplotlib & Seaborn**: Visualización

### Métricas de Evaluación

- **ROC-AUC**: Capacidad discriminativa del modelo
- **Precision**: Proporción de predicciones positivas correctas
- **Recall**: Proporción de casos positivos detectados
- **F1-Score**: Media armónica de precision y recall
- **Matriz de Confusión**: Análisis detallado de errores

---

**Proyecto desarrollado con fines educativos**
**Fecha**: 2025
**Dataset**: Telco Customer Churn (Kaggle)
